In [1]:
import pandas as pd

import json
import glob
import os
import re

In [2]:
base = "/amd.home/home/i11/zeitz/experiments/chpot/"
paths = glob.glob(base + "applications/*.json")
data = [json.load(open(path)) for path in paths]

In [3]:
queries = pd.DataFrame.from_records([{ 
    'exp': exp['experiment'], 
    'graph': run['args'][1],
    **algo } 
    for run in data for exp in run.get('experiments', []) for algo in exp['algo_runs'] if algo.get('algo') == 'CH Potentials Query'])

queries = queries.append(pd.DataFrame.from_records([{ 
    'exp': run['program'], 
    'graph': run['args'][1],
    **algo } 
    for run in data for algo in run.get('algo_runs', []) if algo.get('algo') in ['CH Potentials Query', 'CH Potentials TD Query']]))

In [4]:
dijkstra_queries = pd.DataFrame.from_records([{ 
    'exp': exp['experiment'], 
    'graph': run['args'][1],
    **algo } 
    for run in data for exp in run.get('experiments', []) for algo in exp['algo_runs'] if algo.get('algo') == 'Dijkstra Query'])

dijkstra_queries = dijkstra_queries.append(pd.DataFrame.from_records([{ 
    'exp': run['program'], 
    'graph': run['args'][1],
    **algo } 
    for run in data for algo in run.get('algo_runs', []) if algo.get('algo') in ['Dijkstra Query', 'TD Dijkstra Query']]))

In [5]:
runtime_pattern = re.compile(".*Needed (\\d+)musec\\..*")

def parse_contraction_output(path):
    stats = { 'preprocessing_running_time_s': 0.0 }

    with open(path, 'r') as f:
        for line in f:
            if not 'graph' in stats:
                stats['graph'] = line.strip()
            else:
                match = runtime_pattern.match(line)
                if match:
                    stats['preprocessing_running_time_s'] += int(match[1]) / 1000000

    return stats

preprocessing = pd.DataFrame.from_records([parse_contraction_output(path) for path in glob.glob(base + "preprocessing/*.out")])

In [6]:
queries['affected'] = queries['lower_bound'] != queries['result']
queries['increase'] = queries['result'] / queries['lower_bound']

In [7]:
table = queries.groupby(['graph', 'exp']).agg(
    running_time_ms=('running_time_ms', 'mean'), 
    num_pot_evals=('num_pot_evals', 'mean'), 
    num_relaxed_arcs=('num_relaxed_arcs', 'mean'),
    increase=('increase', 'mean'),
    affected=('affected', 'sum'),
    size=('affected', 'count'))
table['affected'] = table['affected'] * 100 / table['size']
table = table[table.columns[0:-1]]
table['num_relaxed_arcs'] = table['num_relaxed_arcs'] / 1000
table['num_pot_evals'] = table['num_pot_evals'] / 1000
table = table.join(preprocessing.groupby('graph').mean())
table = table.join(dijkstra_queries.groupby(['graph', 'exp']).agg(dijkstra_running_time_ms=('running_time_ms', 'mean')))
#table = table.round(1)
print(table.to_latex())
table

\begin{tabular}{llrrrrrrr}
\toprule
                                         &            &  running\_time\_ms &  num\_pot\_evals &  num\_relaxed\_arcs &  increase &  affected &  preprocessing\_running\_time\_s &  dijkstra\_running\_time\_ms \\
graph & exp &                  &                &                   &           &           &                               &                           \\
\midrule
/algoDaten/graphs/cleaned\_td\_road\_data/de/day/dido/ & chpot\_td &         4.341473 &       7.621476 &         30.077662 &  1.030244 &     98.23 &                     59.086588 &                604.205920 \\
/algoDaten/graphs/cleaned\_td\_road\_data/ptv17-eur-car/day/di/ & chpot\_td &        85.371324 &     103.096165 &        500.442628 &  1.038714 &     99.48 &                    292.520587 &               3245.881700 \\
/algoDaten/zeitz/roadgraphs/europe/ & fastest\_times\_10 &        50.767913 &     111.816654 &        390.435148 &  1.028436 &     96.28 &                    273.

running_time_ms  \
graph                                              exp                                 
/algoDaten/graphs/cleaned_td_road_data/de/day/d... chpot_td                 4.341473   
/algoDaten/graphs/cleaned_td_road_data/ptv17-eu... chpot_td                85.371324   
/algoDaten/zeitz/roadgraphs/europe/                fastest_times_10        50.767913   
                                                   fastest_times_2          6.696532   
                                                   random_times_10          0.975721   
                                                   random_times_2           0.970561   
/algoDaten/zeitz/roadgraphs/osm_ger_rel/           chpot_live             107.523830   
                                                   chpot_turns             38.234487   
                                                   no_highways            315.260351   
                                                   no_tunnels              29.906702   

                                                                     num_pot_evals  \
graph                                              exp                               
/algoDaten/graphs/cleaned_td_road_data/de/day/d... chpot_td               7.621476   
/algoDaten/graphs/cleaned_td_road_data/ptv17-eu... chpot_td             103.096165   
/algoDaten/zeitz/roadgraphs/europe/                fastest_times_10     111.816654   
                                                   fastest_times_2       13.683046   
                                                   random_times_10        1.127394   
                                                   random_times_2         1.111212   
/algoDaten/zeitz/roadgraphs/osm_ger_rel/           chpot_live           218.899497   
                                                   chpot_turns           85.649110   
                                                   no_highways          661.362835   
                                                   no_tunnels            60.373990   

                                                                     num_relaxed_arcs  \
graph                                              exp                                  
/algoDaten/graphs/cleaned_td_road_data/de/day/d... chpot_td                 30.077662   
/algoDaten/graphs/cleaned_td_road_data/ptv17-eu... chpot_td                500.442628   
/algoDaten/zeitz/roadgraphs/europe/                fastest_times_10        390.435148   
                                                   fastest_times_2          42.021788   
                                                   random_times_10           2.908221   
                                                   random_times_2            2.864245   
/algoDaten/zeitz/roadgraphs/osm_ger_rel/           chpot_live              801.182286   
                                                   chpot_turns             178.109398   
                                                   no_highways            2465.660474   
                                                   no_tunnels              224.124574   

                                                                     increase  \
graph                                              exp                          
/algoDaten/graphs/cleaned_td_road_data/de/day/d... chpot_td          1.030244   
/algoDaten/graphs/cleaned_td_road_data/ptv17-eu... chpot_td          1.038714   
/algoDaten/zeitz/roadgraphs/europe/                fastest_times_10  1.028436   
                                                   fastest_times_2   1.006781   
                                                   random_times_10   1.000056   
                                                   random_times_2    1.000007   
/algoDaten/zeitz/roadgraphs/osm_ger_rel/           chpot_live        1.137614   
                                                   chpot_turns       1.058078   
                                                   no_highways       1.392460   
                   